In [1]:
import time
import pywikibot
import mwparserfromhell
import csv
import json
import re
import xml.sax
from bs4 import BeautifulSoup
import requests

In [61]:
site = pywikibot.Site("id", "wikipedia")
art = pywikibot.Page(site, 'Raden Mattaher')


wikicode = mwparserfromhell.parse(art.get())
matches = wikicode.filter_templates(matches = "Infobox")
properties = {param.name.strip(): param.value
                                for param in matches[0].params}
print(properties)

{'name': ' Raden Mattaher\n', 'image': ' Raden Mattaher.jpg\n', 'imagesize': ' 200px\n', 'alt': ' \n', 'caption': ' \n', 'birth_name': ' \n', 'birth_date': ' 1871\n', 'birth_place': ' Dusun Sekamis, [[Kesultanan Jambi]]\n', 'death_date': ' {{death date and age|1907|9|10|1871|df=y}}\n', 'death_place': ' Dusun Muaro Jambi, [[Keresidenan Jambi]], [[Hindia Belanda]]\n', 'other_names': ' Raden Mohammad Tahir, Pangeran Mattahir\n', 'known_for': " Pejuang Kemerdekaan [[Indonesia]] dari [[Jambi]], dan [[Pahlawan Nasional Indonesia]]<ref>[https://m.republika.co.id/amp/qjk8kb335 Raden Mattaher, Singo Kumpeh yang Jadi Pahlawan Nasional] ''republika'' 10 November 2020</ref>\n", 'occupation': ' \n', 'religion': ' \n', 'spouse': ' \n', 'children': ' \n', 'parents': ' Raden Kusin (ayah)<br> Ratumas Esa (ibu)\n', 'relatives': ' [[Sultan Thaha Syaifuddin]] (kakek)\n'}


In [62]:
print(matches)

["{{Infobox person\n| name               = Raden Mattaher\n| image              = Raden Mattaher.jpg\n| imagesize          = 200px\n| alt                = \n| caption            = \n| birth_name         = \n| birth_date         = 1871\n| birth_place        = Dusun Sekamis, [[Kesultanan Jambi]]\n| death_date         = {{death date and age|1907|9|10|1871|df=y}}\n| death_place        = Dusun Muaro Jambi, [[Keresidenan Jambi]], [[Hindia Belanda]]\n| other_names        = Raden Mohammad Tahir, Pangeran Mattahir\n| known_for          = Pejuang Kemerdekaan [[Indonesia]] dari [[Jambi]], dan [[Pahlawan Nasional Indonesia]]<ref>[https://m.republika.co.id/amp/qjk8kb335 Raden Mattaher, Singo Kumpeh yang Jadi Pahlawan Nasional] ''republika'' 10 November 2020</ref>\n| occupation         = \n| religion           = \n| spouse             = \n| children           = \n| parents            = Raden Kusin (ayah)<br> Ratumas Esa (ibu)\n| relatives          = [[Sultan Thaha Syaifuddin]] (kakek)\n}}"]


In [59]:
for key,value in properties.items():
    print(key,":",value)
    print("===========")

name :  Sultan Thaha Syaifuddin

title :  

titletext :  

image :  Sultan Thaha Syaifuddin.jpg

caption :  

succession :  [[Kesultanan Jambi#Daftar Penguasa|Sultan Jambi]] ke-20

reign :  1855 - 26 April 1904

date1 :  

date2 :  

reign-type :  

coronation :  

predecessor :  Sultan Abdurrahman Nazaruddin

successor :  

royal house :  

father :  Sultan Muhammad Fachruddin

issue :  

birth_date :  {{birth year|1816}}

birth_place :  [[Kota Jambi|Tanah Pilih]], [[Kesultanan Jambi]]

death_date :  {{death date|1904|4|26}} (87 atau 88 tahun)

death_place :  Betung Bedarah, [[Tebo Ilir, Tebo|Tebo Ilir]], [[Tebo]]



In [103]:
wikicode = mwparserfromhell.parse(properties['commander1'])
print(str(properties['commander1']))
temp = re.sub("<br>","{{br}}",str(properties['commander1']))
print(re.sub("{{br}}","\n",temp))

 {{flagicon|Belanda}} [[Johan Harmen Rudolf Köhler|Johan Köhler]]{{KIA}}
<br>{{flagicon|Belanda}} [[Jan van Swieten]] 
<br>{{flagicon|Belanda}} [[Johannes Ludovicius Jakobus Hubertus Pel|Johannes Pel]]{{KIA}}
<br>{{flagicon|Belanda}} [[Karel van der Heijden]]{{WIA}}
<br>{{flagicon|Belanda}} [[Henry Demmeni]]{{DOW}}
<br>{{flagicon|Belanda}} [[Jan Jacob Karel de Moulin]]{{KIA}}
<br>{{flagicon|Belanda}} [[G.C.E. van Daalen (1863-1930)|Gotfried van Daalen]]
<br>{{flagicon|Belanda}} [[Johan Cornelis van der Wijck]]
{{flagicon|Belanda}} [[Johannes Benedictus van Heutsz]]

 {{flagicon|Belanda}} [[Johan Harmen Rudolf Köhler|Johan Köhler]]{{KIA}}

{{flagicon|Belanda}} [[Jan van Swieten]] 

{{flagicon|Belanda}} [[Johannes Ludovicius Jakobus Hubertus Pel|Johannes Pel]]{{KIA}}

{{flagicon|Belanda}} [[Karel van der Heijden]]{{WIA}}

{{flagicon|Belanda}} [[Henry Demmeni]]{{DOW}}

{{flagicon|Belanda}} [[Jan Jacob Karel de Moulin]]{{KIA}}

{{flagicon|Belanda}} [[G.C.E. van Daalen (1863-1930)|Gotfried 

# military conflict

In [88]:
class Infobox():
    def __init__(self,title):
        self.title = title
        site = pywikibot.Site("id", "wikipedia")
        art = pywikibot.Page(site, self.title)
    

        wikicode = mwparserfromhell.parse(art.get())
        matches = wikicode.filter_templates(matches = self.type)
        self.infobox = {param.name.strip(): re.sub("\[\[File:[^\]]*\]\]","",re.sub("\[\[Berkas:[^\]]*\]\]","",str(param.value)))
                            for param in matches[0].params}
#         print(self.infobox)
        self.clean_infobox = self.clean_data()
    def clean_link(self,key):
        return self.filter_wikilinks(self.infobox.get(key,""))
    def clean_text(self,key):
        return self.text(self.infobox.get(key,""))
    def clean_list(self,key):
        return self.list(self.infobox.get(key,""))
    def clean_list2(self,key):
        return self.list2(self.infobox.get(key,""))
    def clean_br(self,key):
        temp = self.infobox.get(key,"")
        temp = re.sub("{{br}}","<br>",str(self.infobox.get(key,"")))
        temp = re.sub("<br />","<br>",temp)
        temp = re.sub("<br>","\n",temp)
        temp = temp.split("\n")
        res = []
        for i in temp:
            if i != "" :
                res.append(self.text(i.strip()))
        return res
    def clean_template(self,key):
        wikitext = self.infobox.get(key,"")
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code(keep_template_params=True).strip())
    def strip_template(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code(keep_template_params=True).strip())
    def filter_wikilinks(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        link = wikicode.filter_wikilinks()
        temp = []
        for i in link:
            temp.append(str(i.title))
        return temp
    def text(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        return re.sub("\n","",wikicode.strip_code().strip())
    def list(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="ubl")
        temp = []
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.text(i.value)))
        template = wikicode.filter_templates(matches="bulleted list")
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.text(i.value)))
        return temp
    def list2(self,wikitext):
        wikicode = mwparserfromhell.parse(wikitext)
        template = wikicode.filter_templates(matches="ubl")
        temp = []
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.strip_template(i.value)))
        template = wikicode.filter_templates(matches="bulleted list")
        if(len(template) > 0):
            for i in template[0].params :
                temp.append(str(self.strip_template(i.value)))
        return temp
    

In [41]:
class Infobox_military_conflict(Infobox):
    def __init__(self,title):
        self.type = "Infobox Military Conflict"
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        temp['conflict'] = self.clean_text('conflict')
        temp['partof'] = self.clean_text('partof')
        temp['image'] = self.clean_text('image')
        temp['caption'] = self.clean_text('caption')
        temp['date'] = self.clean_text('date')
        temp['place'] = self.clean_link('place')
        temp['casus'] = self.clean_text('casus')
        temp['teritory'] = self.clean_text('teritory')
        temp['result'] = self.clean_text('result')
        temp['combatant1'] = self.clean_list("combatant1")
        if(len(temp['combatant1']) == 0):
            temp['combatant1'] = self.clean_br('combatant1')
        temp['combatant2'] = self.clean_list("combatant2")
        if(len(temp['combatant2']) == 0):
            temp['combatant2'] = self.clean_br('combatant2')
        temp['commander1'] = self.clean_list("commander1")
        if(len(temp['commander1']) == 0):
            temp['commander1'] = self.clean_br('commander1')
        temp['commander2'] = self.clean_list("commander2")
        if(len(temp['commander2']) == 0):
            temp['commander2'] = self.clean_br('commander2')
        temp['units1'] = self.clean_list("units1")
        if(len(temp['units1']) == 0):
            temp['units1'] = self.clean_br('units1')
        temp['units2'] = self.clean_list("units2")
        if(len(temp['units2']) == 0):
            temp['units2'] = self.clean_br('units2')
        temp['strength1'] = self.clean_text('strength1')
        temp['strength2'] = self.clean_text('strength1')
        temp['casualties1'] = self.clean_text('casualties1')
        temp['casualties2'] = self.clean_text('casualties2')
        temp['notes'] = self.clean_text('notes')
        return temp
    

In [58]:
diponegoro = Infobox_military_conflict("Perang Diponegoro")
print(diponegoro.clean_infobox)

{'conflict': 'Perang Diponegoro', 'partof': '', 'image': '300px', 'caption': 'Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh', 'date': '21 Juli 1825 - 9 Februari 1830 1840', 'place': '[[Yogyakarta]],[[Surakarta]],[[Jawa Tengah]],[[Jawa Timur]]', 'casus': 'Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6', 'teritory': '', 'result': 'Kemenangan BelandaPenangkapan dan pengasingan Pangeran Diponegoro ke MakassarToby Alice Volkman: Sulawesi: island crossroads of Indonesia, Passport Books, 1990, ISBN 0844299065,  page 73.', 'combatant1': '[[Imperium Belanda|Belanda]],[[Kesultanan Yogyakarta]],[[Pasukan Tulungan]]', 'combatant2': 'Pasukan Jawa/Diponegoro', 'commander1': '[[Hendrik Merkus de Kock|Hendrik M. de Kock]],[[Hermanus W. Dotulong]]', 'commander2': '[[Diponegoro]],[[Sentot Prawirodirdjo]],[[Kiai Madja]],[[Nyi Ageng Serang]],[[Pakubuwana VI]],[[Tumenggung Prawirodigdoyo]]', 'strength1': '50.000', 'strength2': '50.000', 'casualties1': '15.000

In [180]:
infobox = json.load(open("infobox\\json\\v1Infobox_Military_Conflict1.json"))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_military_conflict(i['entity']))
print(len(daftar_infobox))

11
11


In [181]:
for i in daftar_infobox :
#     print(i.infobox['place'])
#     print("-----------------")
    for j,k in i.clean_infobox.items():
        print(j,":",k)
    print("################")

conflict : Perang Diponegoro
partof : 
image : 
caption : Lukisan Peristiwa Penangkapan Pangeran Diponegoro oleh Raden Saleh
date : 21 Juli 1825 - 9 Februari 1830 1840
place : ['Yogyakarta', 'Surakarta', 'Jawa Tengah', 'Jawa Timur']
casus : Jalan yang dibangun Belanda melintasi makam leluhur Pangeran Diponegoro 6
teritory : 
result : Kemenangan BelandaPenangkapan dan pengasingan Pangeran Diponegoro ke MakassarToby Alice Volkman: Sulawesi: island crossroads of Indonesia, Passport Books, 1990, ISBN 0844299065,  page 73.
combatant1 : ['Belanda', 'Kesultanan Yogyakarta', 'Pasukan Tulungan']
combatant2 : ['Pasukan Jawa/Diponegoro']
commander1 : ['Hendrik M. de Kock', 'Hermanus W. Dotulong']
commander2 : ['Diponegoro', 'Sentot Prawirodirdjo', 'Kiai Madja', 'Nyi Ageng Serang', 'Pakubuwana VI', 'Tumenggung Prawirodigdoyo']
units1 : []
units2 : []
strength1 : 50.000
strength2 : 50.000
casualties1 : 15.000
casualties2 : Serdadu Jawa:20,000 tewas dalam perangClodfelter, Michael, Warfare and Armed

In [148]:
string = "{{flagicon|Belanda}} [[Johannes Ludovicius Jakobus Hubertus Pel|Johannes Pel]]{{KIA}}"
print(re.sub("\{\{[^\}]*\}\}","",string))

['{{flagicon|Belanda}}', '{{KIA}}']


In [182]:
with open("infobox\\clean.json",'w') as file :
    out = []
    for i in daftar_infobox :
        out.append(i.clean_infobox)
    file.write(json.dumps(out))

# Infobox person

In [88]:
class Infobox_person(Infobox):
    def __init__(self,title):
        self.type = "Infobox person"
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        temp['name'] = self.clean_text('name')
        temp['image'] = self.clean_text('image')
        temp['birth_place'] = self.clean_link('birth_place')
        temp['death_place'] = self.clean_link('death_place')
        temp['death_cause'] = self.clean_text('death_cause')
#         temp['resting_place_coordinates'] = self.clean_text('resting_place_coordinates')
        temp['monuments'] = self.clean_text('monuments')
        temp['ethnicity'] = self.clean_text('ethnicity')
        temp['occupation'] = self.clean_text('occupation')
        temp['religion'] = self.clean_text("religion")
        temp['denomination'] = self.clean_text("denomination")
        temp['parents'] = self.clean_br("parents")
        temp['awards'] = self.clean_text("awards")
        temp['caption'] = self.clean_text("caption")
        temp['nationality'] = self.clean_text("nationality")
        temp['spouse'] = self.clean_text('spouse')
        temp['birth_date'] = self.clean_text('birth_date')
        if temp['birth_date'] == "":
            temp['birth_date'] = self.clean_template('birth_date')
        temp['death_date'] = self.clean_text('death_date')
        if temp['death_date'] == "":
            temp['death_date'] = self.clean_template('death_date')
        temp['other_names'] = self.clean_text('other_names')
        temp['known_for'] = self.clean_text('known_for')
        temp['burial_place'] = self.clean_text('burial_place')
        temp['pre-nominals'] = self.clean_text('pre-nominals')
        temp['birth_name'] = self.clean_text('birth_name')
        temp['birthname'] = self.clean_text('birthname')
        temp['restingplace'] = self.clean_link('restingplace')
        temp['yearsactive'] = self.clean_text('yearsactive')
        temp['years_active'] = self.clean_text('years_active')
        temp['resting_place'] = self.clean_text('resting_place')
        
        temp['spouse-type'] = self.clean_text('spouse-type')
#         temp['module'] = self.clean_text('module')
        temp['title'] = self.clean_text('title')
        temp['father'] = self.clean_text('father')
        temp['house'] = self.clean_text('house')
        temp['partner'] = self.clean_text('partner')
        temp['mother'] = self.clean_text('mother')
        temp['website'] = self.clean_text('website')
        temp['relatives'] = self.clean_text('relatives')
        return temp

In [89]:
infobox = json.load(open("infobox\\json\\v1Infobox_person24.json"))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_person(i['entity']))
print(len(daftar_infobox))

11
11


In [82]:
property = set()
for i in daftar_infobox:
    for key,value in i.infobox.items():
        if re.sub("\s","",str(value)) != "" :
            property.add(key)
print(len(property))

for i in property :
    print(i)

41
birth_name
spouse-type
ethnicity
religion
image
alt
image_size
module
burial_place
birth_date
birth_place
pre-nominals
nationality
death_place
birthname
title
imagesize
other_names
father
death_cause
occupation
years_active
caption
restingplace
house
partner
awards
parents
name
spouse
denomination
known_for
monuments
mother
yearsactive
website
relatives
death_date
resting_place_coordinates
signature_type
resting_place


In [101]:
for i in daftar_infobox :
    print(i.infobox.get('signature_type',""))
    print("-----------------")
    print(i.clean_infobox['signature_type'])
#     for j,k in i.clean_infobox.items():
#         print(j,":",k)
    print("################")


-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################
 Tanda tangan

-----------------
Tanda tangan
################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################


# infobox office holder

In [84]:
def get_property(source,type):
    infobox = json.load(open(source))
    res = set()
    for i in infobox :
        title = i['entity']
        site = pywikibot.Site("id", "wikipedia")
        art = pywikibot.Page(site, title)
    

        wikicode = mwparserfromhell.parse(art.get())
        matches = wikicode.filter_templates(matches = type)
        if(len(matches) > 0):
            property = {param.name.strip(): re.sub("\[\[File:[^\]]*\]\]","",re.sub("\[\[Berkas:[^\]]*\]\]","",str(param.value)))
                                for param in matches[0].params}
            for key,value in property.items():
                if re.sub("\s","",str(value)) != "" :
                    res.add(key)

    print(len(res))

    for i in res :
        print(i)
    return res

In [105]:
get_property("infobox\\json\\v1Infobox_person24.json","Infobox person")

41
birth_name
spouse-type
ethnicity
religion
image
alt
image_size
module
burial_place
birth_date
birth_place
pre-nominals
nationality
death_place
birthname
title
imagesize
other_names
father
death_cause
occupation
years_active
caption
restingplace
house
partner
awards
parents
name
spouse
denomination
known_for
monuments
mother
yearsactive
website
relatives
death_date
resting_place_coordinates
signature_type
resting_place


In [163]:
class Infobox_Officeholder(Infobox):
    def __init__(self,title,property):
        self.type = "Infobox Officeholder"
        self.property = property
        Infobox.__init__(self,title)
    def clean_data(self):
        temp = {}
        for i in self.property :
            temp[i] = self.clean_text(i)
        temp['branch'] = self.clean_list("branch")
        temp['monarch3'] = self.clean_br("monarch3")
        temp['president3'] = self.clean_br("president3")
        temp['allegiance'] = self.clean_list2('allegiance')
        if len(temp['allegiance']) == 0 :
            temp['allegiance'] = self.clean_text('allegiance')
        temp['predecessor'] = self.clean_br('predecessor')
        temp['monarch'] = self.clean_br("monarch")
        temp['death_date'] = self.clean_template('death_date')
        temp['parents'] = self.clean_br('parents')
        temp['president2'] = self.clean_br("president2")
        temp['birth_date'] = self.clean_template('birth_date')
        temp['monarch2'] = self.clean_br("monarch2")
        temp['spouse'] = self.clean_br('spouse')
        temp['children'] = self.clean_br('children')
        temp['successor'] = self.clean_br('successor')
        temp['president'] = self.clean_br("president")
        if temp['battles'] == "" :
            temp['battles'] = self.clean_list('battles')
        #death_place,birth_place
        return temp

In [6]:
source = "infobox\\json\\v1Infobox_Officeholder37.json"
type = "Infobox Officeholder"
daftar_property = get_property(source,type)


85
successor5
primeminister3
order4
successor1
order
serviceyears
succeeding
predecessor5
monarch3
partner
president3
office
office2
allegiance
honorific-prefix
name
predecessor
primeminister1
monarch
alma_mater
unit
1
term_end
death_date
death_place
co-leader
predecessor2
term_start1
term_end1
birth_place
footnotes
birthname
parents
title
term_end2
rank
image
successor4
president4
president2
branch
imagesize
term_start3
primeminister
governor3
predecessor1
nationality
predecessor4
successor3
birth_date
term_start4
monarch2
governor
term_end5
caption
order3
office4
predecessor3
term_end3
signature
term_start2
otherparty
office3
religion
term_start
office5
term_start5
successor2
order5
spouse
office1
term_end4
president1
children
successor
primeminister2
awards
president
death_cause
Term_start5
profession
relations
battles
order2
primeminister4


In [164]:
infobox = json.load(open(source))
print(len(infobox))
daftar_infobox = []
for i in infobox :
    daftar_infobox.append(Infobox_Officeholder(i['entity'],daftar_property))
print(len(daftar_infobox))

17
17


In [167]:
key = 'primeminister4'
for i in daftar_infobox :
    print(i.infobox.get(key,""))
    print("-----------------")
    print(i.clean_infobox[key])
#     for j,k in i.clean_infobox.items():
#         print(j,":",k)
    print("################")


-----------------

################

-----------------

################

-----------------

################

-----------------

################
 [[Mohammad Hatta]]

-----------------
Mohammad Hatta
################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################

-----------------

################
